In [0]:
!pip install nltk
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import math

You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
movie_data = sc.textFile('/FileStore/tables/assignment1/plot_summaries.txt')
movie_data.take(3)

#Current format is MovieID \t plot summary.

Out[6]: ["23890098\tShlykov, a hard-working taxi driver and Lyosha, a saxophonist, develop a bizarre love-hate relationship, and despite their prejudices, realize they aren't so different after all.",
 '31186339\tThe nation of Panem consists of a wealthy Capitol and twelve poorer districts. As punishment for a past rebellion, each district must provide a boy and girl  between the ages of 12 and 18 selected by lottery  for the annual Hunger Games. The tributes must fight to the death in an arena; the sole survivor is rewarded with fame and wealth. In her first Reaping, 12-year-old Primrose Everdeen is chosen from District 12. Her older sister Katniss volunteers to take her place. Peeta Mellark, a baker\'s son who once gave Katniss bread when she was starving, is the other District 12 tribute. Katniss and Peeta are taken to the Capitol, accompanied by their frequently drunk mentor, past victor Haymitch Abernathy. He warns them about the "Career" tributes who train intensively at special 

In [0]:
movie_metadata = sc.textFile('/FileStore/tables/assignment1/movie_metadata.tsv')
movie_metadata = movie_metadata.map(lambda line: line.split('\t'))
movie_metadata.take(5)

Out[9]: [['11389149',
  '/m/02r9vpx',
  'Fall Guy',
  '1982-10-09',
  '',
  '108.0',
  '{"/m/03_9r": "Japanese Language"}',
  '{"/m/03_3d": "Japan"}',
  '{"/m/0gw5n2f": "Japanese Movies", "/m/01z4y": "Comedy", "/m/03q4nz": "World cinema"}'],
 ['31022942',
  '/m/0gg61sf',
  'Fall Guy',
  '1947-03-15',
  '',
  '',
  '{}',
  '{}',
  '{"/m/02xh1": "Film noir"}']]

In [0]:
"""
Cleaning of the plot summaries using word tokenizer and stop word removal.
"""

stop_words = set(stopwords.words("english")).union((',','.'))

refined_movie_data = movie_data.map(lambda x: (x.split('\t')[0], [w for w in word_tokenize(x.split('\t')[1]) if not w.lower() in stop_words]))

#New format - (movieId, refinedPlot)

Out[10]: [('31022942',
  ['memory',
   'night',
   'question',
   'clues',
   'man',
   'tries',
   'prove',
   'murder',
   'attractive',
   'woman'])]

In [0]:
"""
Converting the rdd to new format - ((doc_id,word),1)
"""

dwc = refined_movie_data.flatMap(lambda x: [((x[0], w), 1) for w in x[1]])
dwc.take(4)


Out[27]: [(('23890098', 'Shlykov'), 1),
 (('23890098', 'hard-working'), 1),
 (('23890098', 'taxi'), 1),
 (('23890098', 'driver'), 1)]

In [0]:
"""
Converting the rdd to new format - (word,(document, count)) (We have calculated the term-frequencies)
"""

tdwc = dwc.reduceByKey(lambda x,y: x+y)
tf = tdwc.map(lambda x: (x[0][1],(x[0][0],x[1])))
tf.take(5)




Out[28]: [('speaking', ('20663735', 1)),
 ('inform', ('5272176', 1)),
 ('fact', ('5272176', 1)),
 ('voices', ('1952976', 1)),
 ('Miss', ('24225279', 5))]

In [0]:
"""
Converting the rdd to new format - (word,(doc,count,1))
"""


dc = tdwc.map(lambda x: (x[0][1],(x[0][0],x[1],1)))
dc.take(5)

#dc New format - (word,(doc,count,1))

Out[29]: [('speaking', ('20663735', 1, 1)),
 ('inform', ('5272176', 1, 1)),
 ('fact', ('5272176', 1, 1)),
 ('voices', ('1952976', 1, 1)),
 ('Miss', ('24225279', 5, 1))]

In [0]:
"""
New format - (word, 1) 1 per doc. We need to do this so that we can count number of docs the word is present in.
"""

idfc = dc.map(lambda x:(x[0],x[1][2]))
idfc.take(2)
#idfc New format - (word, 1) 1 per doc.


Out[31]: [('clean', 1), ('give', 1)]

In [0]:
"""
Calculate the inverse document frequency after reduce, we use num_doc (total number of docs in the corpus)
"""

num_doc = movie_data.count()
idf=idfc.reduceByKey(lambda x,y:x+y).map(lambda x: (x[0],math.log10(num_doc/x[1])))
idf.take(5)

#New format - (word, idf)

Out[32]: [('attempt', 1.049485009095479),
 ('marriage', 1.0987720641893475),
 ('would', 0.90781724503325),
 ('arrangement', 2.375981962751792),
 ('feels', 1.2855614152483545)]

In [0]:
"""
Joining the two rdds on key word and then transform into (doc_id, (word, tf-idf))
"""
tf_idf = tf.join(idf)
tf_idf = tf_idf.map(lambda x: (x[1][0][0],(x[0],x[1][0][1]*x[1][1])))
tf_idf.take(5)



Out[33]: [('476640', ('months', 1.4000598778970554)),
 ('223902', ('months', 1.4000598778970554)),
 ('5179684', ('months', 1.4000598778970554)),
 ('4954774', ('months', 1.4000598778970554)),
 ('2804704', ('months', 1.4000598778970554))]

In [0]:
"""
Create functions to help us do the operations.
"""


def oneword_query_search(query):
    global tf_idf
    return get_movies(tf_idf.filter(lambda x: x[1][0] == query).sortBy(lambda x: -x[1][1]).take(10))


def cosine_similiarity(doc1,doc2):
    doc1 = set(doc1)
    # form a set containing keywords of both strings 
    rvector = doc1.union(doc2)
    l1 = []; l2 = []
    
    for w in rvector:
        if w in doc1: l1.append(1) 
        else: l1.append(0)
        if w in doc2: l2.append(1)
        else: l2.append(0)
    c = 0

    for i in range(len(rvector)):
            c+= l1[i]*l2[i]
    cosine = c / float((sum(l1)*sum(l2))**0.5)
    return cosine

def get_cosine_map(query):
    global refined_movie_data
    doc = {w for w in word_tokenize(query) if not w.lower() in stop_words}
    return get_movies(refined_movie_data.map(lambda x: (x[0], cosine_similiarity(x[1],doc))).sortBy(lambda x: -x[1]).take(10))

def get_movies(movies):
    global movie_metadata
    return [movie_metadata.filter(lambda x: x[0] == movie[0]).collect()[0][2] for movie in movies]

def search(query):
    if len(query.split(' ')) == 1:
        return oneword_query_search(query)
    elif len(query.split(' ')) > 1:
        return get_cosine_map(query)
    else:
        return []

In [0]:
queries = sc.textFile('/FileStore/tables/assignment1/test.txt')
query_list = list(queries.collect())
print(query_list)

In [0]:
query_dict = {}

for query in query_list:
    query_dict[query] = search(query)

print(query_dict)    

{'galaxy': ['Stargate: The Ark of Truth', 'Be Forever Yamato', 'Men in Black', 'Krull', 'Starcrash', "Dragon Fist Explosion!! If Gokū Can't Do It, Who Will?", 'Uchū Senkan Yamato: Fukkatsu hen', 'Star Wars Episode V: The Empire Strikes Back', 'Cassiopéia', 'Wishology'], 'wonder': ['Invasion of Astro-Monster', 'Chidambaram', 'The Last Dragon', 'Franklin and the Green Knight', 'Red Desert', 'Twilight Zone: The Movie', 'Kolberg', 'Money No Enough 2', 'Is There a Doctor in the Mouse?', 'Nightmare Beach'], 'war in the forest': ['Fear and Desire', 'Back to the Forest', 'Counterblast', "Stone's War", 'Ulysses', 'Amphitryon', 'Eito Prem', 'Penny and the Pownall Case', 'Last Plane Out', 'Luck of the Navy'], 'deep ocean travel': ['A propósito de Sudán', 'Cast a Long Shadow', 'Cry Macho', 'The Single Girls', 'The Octoroon', 'Pink Paradise', "Don't Bother to Knock", 'Devadoothan', 'Old Bill and Son', 'The Great Heep']}
